In [41]:
#set openai key in .env.local
import os
import pandas as pd
import openai
import numpy as np
import time
from scipy.sparse import load_npz
from sklearn.metrics import precision_score, recall_score, f1_score

pd.set_option('display.max_colwidth', None)



In [42]:
def set_environment_variables_from_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        if line and not line.startswith('#'):  # Ignore empty lines and comments
            key, value = line.split('=', 1)
            os.environ[key.strip()] = value.strip()
file_path = '../../.env.local'
set_environment_variables_from_file(file_path)
openai.api_key = os.getenv('OPENAI_KEY')

In [43]:
def estimate_price(n):
    num_tokens = 900 * n
    # per thousend tokens 0.0001 $
    price_per_thousend = 0.0001
    price = round(num_tokens/1000 * price_per_thousend, 5)

    response = input(f"Estimated price {price} $\nProceed? (y/n): ")
    if response.lower() == 'y':
        print("Price accepted...")
        # Continue with your notebook code here

    elif response.lower() == 'n':
        print("Price declined, interrupting...")
        raise KeyboardInterrupt  # Raises a KeyboardInterrupt to stop execution

    else:
        print("Invalid input. interrupting...")
        raise KeyboardInterrupt
        # Prompt the user again or take appropriate action

In [44]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [45]:
basepath = '../../datasets/preprocessed_datasets/retailrocket/'
product_data = pd.read_pickle(basepath +'item_data_extracted_10k.pkl')
product_data = product_data.reset_index(drop=True)
# product_data = product_data.sample(n=1000, random_state=2).reset_index(drop=True)
display(product_data)

,itemid,categoryid,available,properties,property_values
0,309858,799,0,"[1066, 112, 159, 202, 227, 283, 364, 384, 400, 566, 595, 6, 678, 689, 698, 764, 776, 790, 839, 853, 888, 917, 934, 964, 992, 997]","[n1260.000 424566, 679677, 519769, 125782 n1440.000 822075, 1308244, 1308244 590597 125782 n1440.000 822075, 909382, 726612, n586.800 424566, n619.200 424566, 873905, 1308244, 590597, 150169 274770 79212, 1308244, 1285872, 1081162, n161400.000, 590597, 826838, 125782 n1440.000 822075, 1301157 n1440.000 822075, 638115, n2640.000 1128577, n21.600 404373, n360.000 863188]"
1,89393,491,0,"[0, 1036, 112, 159, 202, 208, 225, 227, 283, 364, 451, 6, 653, 663, 678, 681, 689, 696, 698, 761, 764, 776, 790, 794, 839, 888, 893, 917, 936, 969]","[[n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, 754228, 754228, 754228, 754228, 754228, 754228], 59972, 679677, 519769, 437265 179731 58653 185433, [769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259 769062, 769062 414047 1078259 769062, 769062 414047 1078259 769062, 769062 414047 1078259 769062, 769062 414047 1078259 769062, 769062 414047 1078259 769062], [1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n10200.000, 1066831 n10200.000, 1066831 n10200.000, 1066831 n10200.000, 1066831 n10200.000, 1066831 n10200.000], 780351, 780351 820477 437265 179731 58653 185433 611917 1242195 437265 179731 251307 863335 347067 988697 266119 519519 30603 386817 990716 312815 1242195 31552 1113810 n9984.000 421638 537209 149279 599418 174342 30603 619143 1128577 876603 47299, 125375, [39392, 39392, 39392, 39392, 39392, 39392, 39392, 39392, 39392, 39392, 973211, 973211, 973211, 973211, 973211, 973211], 780351, 1314495 237874 536050 1323749, 1297729 n36.000 n60.000 1178208 n48.000 606827, 820477, 1078259 50164, 150169 435459 16718, 769062, 780351, n1464.000 495356, 1285872, 365413, n61800.000, 820477 437265 179731 58653 185433 n48.000 1187104, 820477, [437265 179731 58653 185433 n48.000 1187104 231473 59972, 437265 179731 58653 185433 n48.000 1187104 231473 59972, 437265 179731 58653 185433 n48.000 1187104 231473, 437265 179731 58653 185433 n48.000 1187104 231473, 437265 179731 58653 185433 n48.000 1187104 231473, 437265 179731 58653 185433 n48.000

In [46]:
product_data['merged_properties'] = [
    {key: value for key, value in zip(row['properties'], row['property_values'])}
    for _, row in product_data.iterrows()
]

# Output the resulting dictionary
display(product_data)


itemid categoryid available  \
0     309858        799         0   
1      89393        491         0   
2      88277       1576         0   
3      90911        179         0   
4      72133       1694         0   
...      ...        ...       ...   
9995  171847        508         0   
9996  271852        624         0   
9997  206063        352         0   
9998   99270       1355         0   
9999  382713       1512         0   

                                                                                                                                                                                                     properties  \
0                                                                             [1066, 112, 159, 202, 227, 283, 364, 384, 400, 566, 595, 6, 678, 689, 698, 764, 776, 790, 839, 853, 888, 917, 934, 964, 992, 997]   
1                                                           [0, 1036, 112, 159, 202, 208, 225, 227, 283, 364, 451, 6, 653, 663, 678, 681, 689, 696, 698, 761, 764, 776, 790, 794, 839, 888, 893, 917, 936, 969]   
2                                                                                      [1010, 1036, 1064, 112, 159, 202, 227, 252, 283, 33, 364, 6, 678, 689, 693, 696, 698, 764, 776, 790, 839, 888, 917, 981]   
3                                                         [112, 159, 19, 202, 213, 227, 243, 28, 283, 364, 38, 434, 441, 46, 480, 575, 6, 678, 689, 698, 713, 764, 776, 790, 810, 839, 888, 917, 960, 961, 994]   
4     [1032, 1037, 1079, 1090, 112, 120, 159, 186, 189, 202, 227, 231, 28, 283, 355, 364, 408, 422, 464, 496, 580, 581, 6, 678, 698, 701, 764, 765, 776, 783, 790, 809, 815, 839, 888, 892, 917, 978, 981, 988]   
...                                                                                                                                                                                                         ...   
9995                                                                          [112, 159, 19, 202, 227, 243, 28, 283, 364, 38, 434, 441, 46, 575, 6, 678, 689, 698, 713, 764, 776, 790, 810, 839, 888, 917, 960]   
9996            [112, 119, 122, 141, 142, 159, 176, 202, 227, 278, 283, 297, 322, 348, 364, 406, 545, 585, 6, 648, 66, 678, 707, 747, 764, 776, 790, 807, 83, 839, 851, 854, 888, 898, 917, 928, 939, 974, 982]   
9997                                                                                                                  [1036, 112, 159, 202, 227, 283, 364, 6, 678, 689, 698, 713, 764, 776, 790, 839, 888, 917]   
9998                  [112, 159, 202, 227, 28, 283, 293, 30, 348, 364, 405, 415, 546, 553, 584, 6, 628, 637, 646, 658, 675, 678, 74, 753, 764, 776, 790, 837, 839, 846, 847, 867, 888, 917, 928, 935, 941, 964]   
9999                                                                                                    [112, 159, 202, 283, 348, 364, 6, 660, 678, 689, 713, 764, 776, 790, 797, 810, 839, 888, 917, 928, 961]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [47]:
def combine_columns(row):
    return  (
        "ID: " + str(row.itemid) + "; " +
        "categoryid: " + str(row.categoryid) + "; " +
        "available: " + row.available + "; " +
        "properties: " + str(row.merged_properties) + "; " 
    )

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

def get_embeddings(texts, model="text-embedding-ada-002"):
    def chunk_list(lst, chunk_size):
        for i in range(0, len(lst), chunk_size):
            yield lst[i:i + chunk_size]

    def embed_chunk(chunk):
        try:
            return openai.Embedding.create(input=chunk, model=model).data
        except openai.error.RateLimitError:
            print("Rate limit exceeded, waiting for 60 seconds...")
            time.sleep(60)  # Wait for 60 seconds
            return embed_chunk(chunk)  # Retry embedding the chunk
        
    # Split the texts into chunks of 1000
    text_chunks = list(chunk_list(texts, 100))
    all_embeddings = []

    # Process each chunk separately
    for chunk in text_chunks:
        embeddings = embed_chunk(chunk)
        all_embeddings.extend(embeddings)

    return all_embeddings

def get_list_from_embeddings(embedding_result):
    list = []
    for entry in embedding_result:
        list.append(entry.embedding)
    return list

In [48]:
# pd.set_option('display.max_colwidth', 100000)
# average_length = product_data_embeddings['combined'].apply(len).max()
# average_length

In [49]:
product_embeddings_path = "openai/product_embeddings_retailrocket.pkl"
recreate_embeddings = False
if not os.path.exists(basepath + product_embeddings_path) or recreate_embeddings:
    product_data_embeddings = product_data[['itemid']].copy()
    product_data_embeddings["combined"] = product_data.apply(lambda row: combine_columns(row), axis=1)
    # print(product_data_embeddings.combined.tolist())
    #print('average length: ' + product_data_embeddings['combined'].apply(len).max())
    estimate_price(product_data_embeddings.shape[0])
    # embed each row separately
    # product_data_embeddings['embedding'] = product_data_embeddings.combined.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
    # batch embed
    product_data_embeddings['embedding'] = get_list_from_embeddings(get_embeddings(product_data_embeddings.combined.tolist()))
    product_data_embeddings.to_pickle(basepath + product_embeddings_path)
else:
    product_data_embeddings = pd.read_pickle(basepath + product_embeddings_path)
#product_data_embeddings = product_data_embeddings.head(10)
display(product_data_embeddings)

itemid  \
0     309858   
1      89393   
2      88277   
3      90911   
4      72133   
...      ...   
9995  171847   
9996  271852   
9997  206063   
9998   99270   
9999  382713   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [50]:
product_data_embeddings.iloc[0].combined

"ID: 309858; categoryid: 799; available: 0; properties: {'1066': 'n1260.000 424566', '112': '679677', '159': '519769', '202': '125782 n1440.000 822075', '227': '1308244', '283': '1308244 590597 125782 n1440.000 822075', '364': '909382', '384': '726612', '400': 'n586.800 424566', '566': 'n619.200 424566', '595': '873905', '6': '1308244', '678': '590597', '689': '150169 274770 79212', '698': '1308244', '764': '1285872', '776': '1081162', '790': 'n161400.000', '839': '590597', '853': '826838', '888': '125782 n1440.000 822075', '917': '1301157 n1440.000 822075', '934': '638115', '964': 'n2640.000 1128577', '992': 'n21.600 404373', '997': 'n360.000 863188'}; "

In [51]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_n_nearest_neighbors(query_embedding, neighbors_embedding, n = 5):
    similarities = [cosine_similarity(query_embedding, neighbor) for neighbor in neighbors_embedding]
    indices = np.flip(np.argsort(similarities)[-n:])
    scores = [similarities[i] for i in indices]
    # Get the n neighbors with the highest similarity values
    n_neighbors = [neighbors_embedding[i] for i in indices]

    return indices, scores


indices, scores = get_n_nearest_neighbors(product_data_embeddings.embedding[0], product_data_embeddings.embedding)
display(indices, scores)

array([   0, 1199, 6812, 7227, 6727])

[1.0,
 0.9902000481432809,
 0.989128534409755,
 0.9883290647181282,
 0.9877318488608059]

In [52]:
product_data_embeddings.iloc[indices].combined

0                                                                                                                                                                                                                      ID: 309858; categoryid: 799; available: 0; properties: {'1066': 'n1260.000 424566', '112': '679677', '159': '519769', '202': '125782 n1440.000 822075', '227': '1308244', '283': '1308244 590597 125782 n1440.000 822075', '364': '909382', '384': '726612', '400': 'n586.800 424566', '566': 'n619.200 424566', '595': '873905', '6': '1308244', '678': '590597', '689': '150169 274770 79212', '698': '1308244', '764': '1285872', '776': '1081162', '790': 'n161400.000', '839': '590597', '853': '826838', '888': '125782 n1440.000 822075', '917': '1301157 n1440.000 822075', '934': '638115', '964': 'n2640.000 1128577', '992': 'n21.600 404373', '997': 'n360.000 863188'}; 
1199                                                                                                                       

In [53]:
filename = 'ground_truth/ground_truth_10k.npz'
ground_truth_csr = load_npz(basepath + filename)

def getMetrics(item_id, recommended_items, n):
    true_positives = 0
    for idx in recommended_items:
        if ground_truth_csr[item_id, idx]:
            true_positives += 1
    y_true = ground_truth_csr[item_id, recommended_items].toarray().ravel()
    y_pred = [1] * len(recommended_items)
    accuracy = y_true.mean()
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    return accuracy, precision, recall, f1

def fix_indices(query_index, indices):
    return [index if index < query_index else index + 1 for index in indices]

def evaluate_coverage(embeddings_df, n=5):
    embeddings = embeddings_df.tolist()
    recommended_items = set()  # Set to store unique recommended items
    total_items = len(embeddings)
    scores_list = []

    accuracies = []
    precisions = []
    recalls = []
    f1s = []

    for i, query_embedding in enumerate(embeddings):
        # Exclude the current query embedding from the neighbors
        neighbors_embeddings = embeddings[:i] + embeddings[i+1:]

        indices, scores = get_n_nearest_neighbors(query_embedding, neighbors_embeddings, n)
        indices = fix_indices(i, indices)
        accuracy, precision, recall, f1 = getMetrics(i, indices, n)
        accuracies.append(accuracy); precisions.append(precision); recalls.append(recall); f1s.append(f1)
    
        recommended_items.update(indices)
        scores_list.extend(scores)
        if i % 100 == 0:
            print(i)

    coverage_score = len(recommended_items) / (total_items - 1)  # Adjusting total_items to account for removal
    overall_acc = np.average(accuracies)
    overall_precision = np.average(precisions)
    overall_recall = np.average(recalls)
    overall_f1 = np.average(f1s)
    coverage_score = len(recommended_items) / (total_items - 1)
    
    return coverage_score, scores_list, overall_acc, overall_precision, overall_recall, overall_f1

# Example usage
coverage_score, confidence_scores, accuracy, precision, recall, f1 = evaluate_coverage(product_data_embeddings.embedding)
print(f"Coverage Score: {coverage_score}")
print(f"Confidence Score: {np.mean(confidence_scores)}; Min Score: {np.min(confidence_scores)}; Max Score: {np.max(confidence_scores)}")
print( f"Accuracy: {accuracy}; Precision: {precision}; Recall: {recall}; F1: {f1}")


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
Coverage Score: 0.8862886288628863
Confidence Score: 0.9904682274443124; Min Score: 0.9699870020866945; Max Score: 0.9977755877757876
Accuracy: 0.3901; Precision: 0.3901; Recall: 0.7324; F1: 0.47792023809523815
